In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()

import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo

  Activating project at `d:\CMU\2024_Spring_Course\OCRL\ocrl-piano\Julia_fmincon_tests`
  No Changes to `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\Julia_fmincon_tests\Project.toml`
    Updating `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\Julia_fmincon_tests\Manifest.toml`
  [d360d2e6] - ChainRulesCore v1.23.0
  [9e997f8a] - ChangesOfVariables v0.1.8


In [3]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [4]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [5]:
# import piano model
model = load_model("cartpole.xml")
data = init_data(model)

MuJoCo Data object

In [6]:

step!(model, data)

println("Initial position: ", data.qpos)
println("Initial velocity: ", data.qvel)
println("model states:",get_physics_state(model, data))

Initial position: [0.0; 0.0;;]
Initial velocity: [0.0; 0.0;;]
model states:[0.0, 0.0, 0.0, 0.0]


## Test DIRCOL: moving index finger left right

In [7]:
function wrapped_mj_step(model, data, x, uk)    
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    step!(model, data) 

    # return updated state k + 1
    xkp1 = zeros(model.nq + model.nv) 
    xkp1 .= get_physics_state(model, data)
    
    return xkp1
end

function cartpole_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf

    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        J += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end
    
    # dont forget terminal cost 
    xt = Z[idx.x[N]]
    J += 0.5*(xt-xg)'*Q*(xt-xg)
    return J 
end
function cartpole_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        # println("xi: ", xi[1:5])
        # print("ui: ", typeof(ui))
        xip1_mujoco = wrapped_mj_step(model, data, xi, ui)
        c[idx.c[i]] = xip1_mujoco - xip1
    end
    # println(typeof(c))
    return c 
end

function cartpole_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 

    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2; cartpole_dynamics_constraints(params, Z)]
end
function cartpole_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [8]:

function cartpole_solve_moving(;verbose=true)
    # instantiate model and data
    model = load_model("cartpole.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.1
    tf = 1.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))


    # Q size only 3 because we track one finger cartesian coordinates
    Q = 1*diagm(ones(model.nq + model.nv))
    R = 0.1*diagm(ones(model.nu))
    Qf = 100*diagm(ones(model.nq + model.nv))

    # indexing 
    idx = create_idx(model.nq + model.nv, model.nu, N)

    # initial and goal state
    xic = [0, pi, 0.1, 0]
    xg = [0, 0, 0, 0]
    
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    
    x_l = -1*Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:(N-1)
        x_l[idx.u[i]] .= -1.0
        x_u[idx.u[i]] .= 1.0
    end
    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)
    # initial guess 
    z0 = 0.1*randn(idx.nz)

    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
    # diff_type = :finite
    
    
    Z = fmincon(cartpole_cost,cartpole_equality_constraints,cartpole_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

cartpole_solve_moving (generic function with 1 method)

In [9]:
X, U, t_vec, params = cartpole_solve_moving(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     2592
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian...

([[7.250099687521798e-33, 3.141592653589793, 0.1, 2.99726911936619e-33], [0.0010097766897686773, 3.141615584032979, 0.10097766897686772, 0.0022930443186029155], [0.0019461980378477468, 3.1414662333632917, 0.09364213480928658, -0.014935066965520938], [0.00292321628452711, 3.141413557976874, 0.09770182466126713, -0.005267538657400737], [0.003948861337958072, 3.1414757567641427, 0.10256450534991568, 0.00621987874254854], [0.004915784404944592, 3.1414000105225925, 0.09669230650636312, -0.007574624599742679], [0.005885454653861365, 3.1413317430098577, 0.09696702529618541, -0.00682675033767908], [0.006842099899399788, 3.1412340604444027, 0.0956645346808602, -0.009768233556156948], [0.007722244427380198, 3.140958540210369, 0.08801435168948257, -0.027552252080910236], [0.008641506061269061, 3.140778379841143, 0.09192665101616576, -0.01801494406631117], [0.004647799179279091, 1.569635171601618, 0.034609506214974516, -0.03439665837846497]], [[0.003323740278761459], [-0.024080061132405114], [0.01

In [10]:
X

11-element Vector{Vector{Float64}}:
 [7.250099687521798e-33, 3.141592653589793, 0.1, 2.99726911936619e-33]
 [0.0010097766897686773, 3.141615584032979, 0.10097766897686772, 0.0022930443186029155]
 [0.0019461980378477468, 3.1414662333632917, 0.09364213480928658, -0.014935066965520938]
 [0.00292321628452711, 3.141413557976874, 0.09770182466126713, -0.005267538657400737]
 [0.003948861337958072, 3.1414757567641427, 0.10256450534991568, 0.00621987874254854]
 [0.004915784404944592, 3.1414000105225925, 0.09669230650636312, -0.007574624599742679]
 [0.005885454653861365, 3.1413317430098577, 0.09696702529618541, -0.00682675033767908]
 [0.006842099899399788, 3.1412340604444027, 0.0956645346808602, -0.009768233556156948]
 [0.007722244427380198, 3.140958540210369, 0.08801435168948257, -0.027552252080910236]
 [0.008641506061269061, 3.140778379841143, 0.09192665101616576, -0.01801494406631117]
 [0.004647799179279091, 1.569635171601618, 0.034609506214974516, -0.03439665837846497]

In [10]:
U

10-element Vector{Vector{Float64}}:
 [0.03930258048384414]
 [-0.04275307734564491]
 [-0.05614929571147601]
 [-0.0345028438922688]
 [0.052669761311953195]
 [-0.023472412642881153]
 [0.007511136586286255]
 [0.03802489554687757]
 [-0.04443802566595274]
 [-0.021326490412473788]

In [11]:
# instantiate model and data
model = load_model("cartpole.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)

nx = model.nq + model.nv + model.na # State vector dimension
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
N = length(t_vec)
reset!(model, data)
states[:,1] = X[1][:]

data.qpos .= X[1][1:2]
data.qvel .= X[1][3:4]
step!(model, data)

for t in 1:(N-1)
    data.ctrl .= U[t][:]
    states[:,t+1] = X[1][:]
end
print(U)

[[0.003323740278761459], [-0.024080061132405114], [0.013436202643971024], [0.016107062079321926], [-0.01925375226787472], [0.0009714802470668994], [-0.0042318473555771055], [-0.025177565716218493], [0.012880153502999437], [-0.050019648752522315]]

In [12]:
init_visualiser()
visualise!(model, data, trajectories = states)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


┌ Warning: The visualiser for MuJoCo.jl is designed to run multi-threaded, but the current Julia session was started with only one thread. Degraded performance will occur. To enable multi-threading, set JULIA_NUM_THREADS to a value greater than 1 before starting Julia.
└ @ VisualiserExt C:\Users\conan0223\.julia\packages\MuJoCo\9JERS\ext\VisualiserExt\VisualiserExt.jl:76
